# 文法特征

第 6 章中我们通过检测文本的特征建立分类器，那些特征可能非常简单，如提取一个单词的最后一个字母等。本章中，我们将探讨特征在建立基于规则的文法中的作用。与第 6 章中自动提取的特征不同，这里我们手动声明词和短语的特征：

In [1]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}

字典对象 kim 和 chase 存储了两组特征值，CAT 代表文法类别，ORTH 代表单词的拼写，还有一些其它面向语义的特征：kim 的 REF 意在给出 kim 的指示物，chase 的 REL 则给出 chase 表示的关系。这样的特征和特征值对被称为**特征结构**。

特征结构包含了各种有关文法实体的信息，我们可以进一步增加属性。例如：对于 chase， 主语扮演施事（agent）角色，而宾语扮演受事（patient）角色。我们添加这些信息：

In [2]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

现在我们来处理句子 Kim chased Lee，我们要“绑定”动词的施事角色给主语，受事角色给宾语，可以通过链接 NP 的 REF 特征来达到这一目的（这里假设了动词左侧和右侧的 NP 分别是主语和宾语）：

In [3]:
sent = 'Kim chased Lee'
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs
        
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print('%-5s => %s' % (k, verb[k]))

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


同样的方法可以适用不同的动词，例如 surprise，不同之处在于这种情况下，主语将扮演来源（source，SRC）角色，宾语将扮演体验者（experiencer，EXP）角色：

In [4]:
surprise = {'CAT': 'V', 'ORTH': 'surprised', 'REL': 'surprise',
            'SRC': 'sbj', 'EXP': 'obj'}

特征结构是非常强大的，接下来我们将分析如何将上下文无关文法扩展到合适的特征结构。

## 句法协议

在英语中，名词通常被标记为单数或附属，例如 this dog 和 these dogs 是符合语法的，而 these dog 和 this dogs 则不是，也就是说名词短语中使用的指示词和名词搭配是由限制的。动词的现在时态也有类似的变化，例如 the dog runs 和 the dogs run。这种同时的变化被称为**协议（agreement）**，下表展示了英语中规则动词的协议规范：

|          | 单数           | 复数     |
|----------|----------------|----------|
| 第一人称 | I run          | we run   |
| 第二人称 | you run        | you run  |
| 第三人称 | he/she/it runs | they run |

让我们看看当我们在一个上下文无关文法中编码这些协议约束会发生什么。我们从一个简单的 CFG 开始：

    S   ->   NP VP
    NP  ->   Det N
    VP  ->   V

    Det  ->  'this'
    N    ->  'dog'
    V    ->  'runs'
    
该文法可以产生句子 this dog runs，然而我们真正想要做的是也能产生 these dogs run，同时阻止不必要的序列如 this dogs run 和 these dog runs：

    S -> NP_SG VP_SG
    S -> NP_PL VP_PL
    NP_SG -> Det_SG N_SG
    NP_PL -> Det_PL N_PL
    VP_SG -> V_SG
    VP_PL -> V_PL

    Det_SG -> 'this'
    Det_PL -> 'these'
    N_SG -> 'dog'
    N_PL -> 'dogs'
    V_SG -> 'runs'
    V_PL -> 'run'
    
在扩展 S 的地方，我们现在有两个产生式，一个覆盖单数主语 NP 和 VP，另一个覆盖复数主语 NP 和 VP，原始文法的所有产生式都有两个与之对应。在小规模文法中这不是什么问题，但是在更大的涵盖了一定量英语成分的文法中，产生式的数量会成爆炸式地增长。

## 使用属性和约束

非正式的语言类别都具有属性，例如：名词具有复数的属性，我们可以用如下符号来表示，它的意思是类别 N 有一个**（文法）特征**叫做 NUM（数字 number 的简写），此特征的值是 pl（复数 plural 的简写）：

    N[NUM=pl]
    
我们可以添加类似的注解给其他类别：

    Det[NUM=sg] -> 'this'
    Det[NUM=pl] -> 'these'

    N[NUM=sg] -> 'dog'
    N[NUM=pl] -> 'dogs'
    V[NUM=sg] -> 'runs'
    V[NUM=pl] -> 'run'
    
当我们在产生式中允许使用特征值变量时，事情变得有趣了起来：

    S -> NP[NUM=?n] VP[NUM=?n]
    NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
    VP[NUM=?n] -> V[NUM=?n]
    
这里我们使用 ?n 来作为 NUM 值得变量，它可以在给定的产生式中被实例化为 sg 或 pl。值得注意的是，在一个产生式中所有的 ?n 需要取同样的值，也就是在 S -> NP VP 中，不管 NP 为特征 NUM 取什么值，VP 必须取同样的值。

我们用树的形式来思考这些特征限制是如何工作的。首先词汇产生式承认下面深度为 1 的树：

![ch09-tree-1.png](resources/ch09-tree-1.png)

接下来通过 NP -> Det N 来产生深度为 2 的树，可以看出后两种情况是不允许的，用顶端节点值为 FAIL 来表示，这是由于它们字数的根节点 NUM 值不同。

![ch09-tree-2.png](resources/ch09-tree-2.png)

再结合扩展 S 的产生式，可以得到 these dogs run 的解析树：

![ch09-tree-3.png](resources/ch09-tree-3.png)

在上面的例子中限定词 Det 有 this 和 these 两种形式，然而英语中的其他限定词对与它们结合的名词数量并不挑剔：

    Det[NUM=sg] -> 'the' | 'some' | 'any'
    Det[NUM=pl] -> 'the' | 'some' | 'any'
    
一个更优雅的写法是保留 NUM 的值为未指定，让它匹配与它结合的任何名词的数量：

    Det[NUM=?n] -> 'the' | 'some' | 'any'
    
事实上我们可以更简单些，在这样的产生式中不给 NUM 任何指定：
    
    Det -> 'the' | 'some' | 'any'
    
下面是一个较为完整的基于特征的文法的例子：

In [5]:
import nltk

nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


文法开头的 % start S 告诉分析器以 S 作为文法的开始符号，同时一个句法类别可以有多个特征，如 IV[TENSE=pres,  NUM=pl]，可以添加任意数量的特征。

我们可以使用 [nltk.load_parse](http://www.nltk.org/_modules/nltk/parse/util.html#load_parser) 函数来加载基于特征的文法：

In [6]:
tokens = 'Kim likes children'.split()
cp = nltk.load_parser('grammars/book_grammars/feat0.fcfg', trace=2)
for tree in cp.parse(tokens):
    print(tree)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: 'sg', ?t: 'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'pl'}
Feature Single Edge Fundame

## 术语

前面我们看到了像 sg 和 pl 这样的特征值，这些简单的值通常被称为**原子**，也就是说它们不能被分解成更小的部分。原子值得一种特殊情况时**布尔值**，也就是说值仅仅指定一个属性是真还是假。例如：我们用布尔特征 AUX 区分**助动词**，如 can、may、will 和 do，就可以写成 AUX=+ 或 AUX=-，有一个广泛采用的缩写约定为 +AUX 和 -AUX。

    V[TENSE=pres, +AUX] -> 'can'
    V[TENSE=pres, +AUX] -> 'may'
    V[TENSE=pres, -AUX] -> 'walks'
    V[TENSE=pres, -AUX] -> 'likes'
    
除了原子值特征以外，特征可能本省就是特征结构的值。例如：我们可以将协议特征组合在一起（如：人称、数量和性别）作为一个类别的不同部分，表示为 AGR，这种情况下，AGR 就是一个**复杂值**，在格式上称为**属性值矩阵**（attribute value matrix，AVM）。

    [POS = N           ]
    [                  ]
    [AGR = [PER = 3   ]]
    [      [NUM = pl  ]]
    [      [GND = fem ]]

当我们有可能使用像 AGR 这样的特征时，我们可以重构前面的文法，使协议特征捆绑在一起：

    S                    -> NP[AGR=?n] VP[AGR=?n]
    NP[AGR=?n]           -> PropN[AGR=?n]
    VP[TENSE=?t, AGR=?n] -> Cop[TENSE=?t, AGR=?n] Adj

    Cop[TENSE=pres,  AGR=[NUM=sg, PER=3]] -> 'is'
    PropN[AGR=[NUM=sg, PER=3]]            -> 'Kim'
    Adj                                   -> 'happy'